In [3]:
%load_ext autoreload
%autoreload 2

In [2]:
import rospy
import numpy as np
import tf_conversions
from robot import Robot

In [5]:
rospy.init_node('test')


# # For single control - cartesian_impedence_controller in franka_example_controller
# robot = Robot(topic='/franka_state_controller/franka_states')

# # For single control - franka-ros-interface
# robot = Robot(topic='/franka_ros_interface/custom_franka_state_controller/robot_state')

# For teleopp
robot = Robot(topic='panda_teleop/follower_state_controller/franka_states')

#### Error recovery for single control follower robot

In [29]:
!rostopic pub -1 /franka_ros_interface/franka_control/error_recovery/goal franka_msgs/ErrorRecoveryActionGoal "{}"

publishing and latching message for 3.0 seconds


#### Record bottelneck poses

In [3]:
poses = []

In [8]:
pose = robot.wait_for_initial_pose()
poses.append(pose)

In [10]:
np.save('./data/poses.npy', poses)

#### Compute the weighted average for the recorded poses

In [11]:
poses = np.load('./data/poses.npy', allow_pickle=True)
pos_ori = []
for i, pose in enumerate(poses):
    bottleneck_pose = tf_conversions.fromMsg(pose)
    pos = bottleneck_pose.p
    ori = bottleneck_pose.M.GetQuaternion()
    pos_ori.append([*pos,  *ori])

pos_ori = np.array(pos_ori)

weights = [1.0, 0.0, 0.0]
# position weighted
pos_w = sum(w * p for p, w in zip(pos_ori[:,:3], weights))
# orientation weighted
A = sum(w * (np.outer(q, q)) for q, w in zip(pos_ori[:,3:], weights))
ori_w = np.linalg.eigh(A)[1][:, -1]

weighted_b_pose = tf_conversions.fromTf((pos_w, ori_w))
tf_conversions.toMsg(weighted_b_pose)

position: 
  x: 0.5113463702810195
  y: 0.002981604851764657
  z: 0.11849767443373138
orientation: 
  x: 0.9997402717888154
  y: 0.005802790763839914
  z: 0.0019010661577782072
  w: -0.021956833342726596

In [12]:
np.save('./data/weighted_bottleneck.npy', tf_conversions.toMsg(weighted_b_pose))

#### Move the robot to the calculated bottleneck pose

In [ ]:
import sys
# adding to the system path
sys.path.insert(0, '/home/prashanth/Thesis/Imitation-Learning/')
from Robot.sawyer import Sawyer

robot = Sawyer()


In [14]:
robot.move_to_pose(weighted_b_pose)


[INFO] [1692640169.557521]: PandaArm: Trajectory controlling complete


##### Alternative to the above if using a impedance controller fron frankla-example-controllers

In [53]:
from geometry_msgs.msg import PoseStamped
link_name = rospy.get_param("/interactive_marker/link_name")
pose_pub = rospy.Publisher('/cartesian_impedance_example_controller/equilibrium_pose', PoseStamped, queue_size=10)

pose_stamped = PoseStamped()
pose_stamped.pose = tf_conversions.toMsg(weighted_b_pose)
pose_stamped.header.frame_id = link_name
pose_stamped.header.stamp = rospy.Time(0)
pose_pub.publish(pose_stamped)

#### Record interaction trajectory for Ergodic exploration

In [107]:
!rostopic pub -1 /panda_teleop/error_recovery/goal franka_msgs/ErrorRecoveryActionGoal "{}"

publishing and latching message for 3.0 seconds


In [6]:
# For teleopp
robot = Robot(topic='panda_teleop/follower_state_controller/franka_states')
ros_rate = rospy.Rate(30)



In [ ]:
robot.wait_for_initial_pose()

In [132]:
import pandas as pd

keys = {
    'B_bw2': [],
    'B_bw5': [],
    'B_bw20': [],
    'B_bw50': [],
    'B_bw100': [],
    'Time_approach_B_bw2': [],
    'Time_approach_B_bw5': [],
    'Time_approach_B_bw20': [],
    'Time_approach_B_bw50': [],
    'Time_approach_B_bw100': [],
    'Time_interaction_B_bw2': [],
    'Time_interaction_B_bw5': [],
    'Time_interaction_B_bw20': [],
    'Time_interaction_B_bw50': [],
    'Time_interaction_B_bw100': [],

}

df = pd.DataFrame(keys)
df.head()


df_row = {
    'B_bw2': None,
    'B_bw5': None,
    'B_bw20': None,
    'B_bw50': None,
    'B_bw100': None,
    'Time_approach_B_bw2': None,
    'Time_approach_B_bw5': None,
    'Time_approach_B_bw20': None,
    'Time_approach_B_bw50': None,
    'Time_approach_B_bw100': None,
    'Time_interaction_B_bw2': None,
    'Time_interaction_B_bw5': None,
    'Time_interaction_B_bw20': None,
    'Time_interaction_B_bw50': None,
    'Time_interaction_B_bw100': None,}

#### For each person

In [153]:
# To be changed
data_path = './data/22/'
filename = 'B_bw100'

import time

trajectory = []
approach_start = time.time()


In [154]:
approach_end = time.time()
df_row['Time_approach_'+str(filename)] = approach_end - approach_start
start = time.time()
try:
    while True:
        pose = robot.wait_for_initial_pose()
        trajectory.append(pose)
        ros_rate.sleep()
except KeyboardInterrupt :
    end = time.time()
    interaction_time = end - start
    print('Stopping recording and saving. \n Length of traj: {}'.format(len(trajectory)))
    # Save the trajectory
    np.save(data_path+'traj_'+filename+'.npy', trajectory)

    bottleneck_pose = tf_conversions.fromMsg(trajectory[0])
    pos = bottleneck_pose.p
    ori = bottleneck_pose.M.GetQuaternion()
    

    df_row[str(filename)] = [*pos,  *ori]
    df_row['Time_interaction_'+str(filename)] = interaction_time
    


Stopping recording and saving. 
 Length of traj: 406


In [157]:
df.loc[len(df)] = df_row
df

/home/prashanth/.local/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:948: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)


,B_bw2,B_bw5,B_bw20,B_bw50,B_bw100,Time_approach_B_bw2,Time_approach_B_bw5,Time_approach_B_bw20,Time_approach_B_bw50,Time_approach_B_bw100,Time_interaction_B_bw2,Time_interaction_B_bw5,Time_interaction_B_bw20,Time_interaction_B_bw50,Time_interaction_B_bw100
0,"[0.5149700536081212, 0.0005244380242898284, 0....","[0.49737543482733365, 0.013753616368447782, 0....","[0.4940946870804789, 0.009318280303112339, 0.1...","[0.5093781709436864, 0.0009600984092425593, 0....","[0.5094123694238064, 0.004342684049733807, 0.0...",22.045897,16.542577,11.536834,10.848669,10.637237,28.969926,8.700356,12.171503,15.484954,13.812867


#### Save demonstration statistics

In [158]:
df.to_csv(data_path+'demo_stats.csv', index=False)

#### Learn Ergodic exploration from the recorded trrajectory

#### Execute obj approach

### Execute obj interaction